In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from math import nan

pd.set_option('display.max_columns', None)


# Aufbereitung

## Monatsbericht


In [2]:
def prepare_monatsbericht(path_origin, path_aim):
    df_1082 = pd.read_csv(path_origin, skiprows =7,
            skipfooter = 4, sep = ';'
            )
    #Adapt column names
    df_1082.columns = ['Steller', 'Produkt', 'Betriebe', 'Betriebe_ep', 'Beschäftigte',
       'Beschäftigte_ep', 'Geleistete Arbeitsstunden', 'Geleistete Arbeitsstunden_ep',
       'Bruttolohn- und -gehaltssumme', 'Bruttolohn- und -gehaltssumme_ep', 'Umsatz', 'Umsatz_ep',
       'Inlandsumsatz', 'Inlandsumsatz_ep', 'Auslandsumsatz', 'Auslandsumsatz_ep',
       'Auslandsumsatz mit der Eurozone', 'Auslandsumsatz mit der Eurozone_ep',
       'Auslandsumsatz mit dem sonstigen Ausland', 'Auslandsumsatz mit dem sonstigen Ausland_ep']
    different_steller = df_1082['Steller'].unique().tolist()
    num_diff_steller = len(different_steller)-12-7-1
    #Add date
    df_1082['month']=0
    for p in range(7):
        k = 2 + (1+(num_diff_steller +1)*12)*p
        for month in range(1,13): 
            for i in range(1,(num_diff_steller +1)):
                df_1082.iloc[k+i+(month-1)*(num_diff_steller +1), -1] = month
    for p in range(7):
        k = 2 + (1+(num_diff_steller +1)*12)*p
        for month in range(0,12): 
            df_1082 =  df_1082.drop(k + month * (num_diff_steller +1),axis=0)
    df_1082['year']=0
    for k in range(7):
        for i in range(1 + 12 * num_diff_steller):
            df_1082.iloc[1 + k* (1 + 12 * num_diff_steller) + i, -1] = k + 2019
    for k in range(7):
        df_1082 =df_1082.drop(1 + k*(1 + 12 * (num_diff_steller+1)),axis = 0)
    df_1082 =df_1082.drop(0,axis = 0)
    df_1082['day']=1
    df_1082['date']=pd.to_datetime(df_1082[['year', 'month', 'day']])

    df_1082 = df_1082.drop('month', axis = 1)
    df_1082 = df_1082.drop('year', axis = 1)
    df_1082 = df_1082.drop('day', axis = 1)
    #Include multiplicities
    for col in ['Betriebe', 'Beschäftigte', 'Geleistete Arbeitsstunden', 'Bruttolohn- und -gehaltssumme', 
            'Inlandsumsatz', 'Auslandsumsatz', 'Auslandsumsatz mit der Eurozone', 'Umsatz',
            'Auslandsumsatz mit dem sonstigen Ausland']:
        df_1082[col]=df_1082[col].replace('...', nan)
        df_1082[col]=df_1082[col].astype('float64')
    df_1082['Geleistete Arbeitsstunden']=df_1082['Geleistete Arbeitsstunden'] * 1000
    df_1082['Bruttolohn- und -gehaltssumme']=df_1082['Bruttolohn- und -gehaltssumme'] * 1000
    df_1082['Umsatz']=df_1082['Umsatz'] * 1000
    df_1082['Inlandsumsatz']=df_1082['Inlandsumsatz'] * 1000
    df_1082['Auslandsumsatz']=df_1082['Auslandsumsatz'] * 1000
    df_1082['Auslandsumsatz mit der Eurozone']=df_1082['Auslandsumsatz mit der Eurozone'] * 1000
    df_1082['Auslandsumsatz mit dem sonstigen Ausland']=df_1082['Auslandsumsatz mit dem sonstigen Ausland'] * 1000
    #Divide df in a df for each different Steller
    different_steller = df_1082['Steller'].unique().tolist()
    df_separated = []
    for steller in different_steller:
        mask = (df_1082['Steller'] == steller)
        df_help = df_1082.loc[mask,:]
        df_help = df_help.drop('Steller', axis=1)
        new_columns = []
        for col in df_help.columns.values[:-1]:
            new_columns.append(col + '_' + steller)
        new_columns.append('date')
        df_help.columns = new_columns
        df_help = df_help.set_index('date')
        df_separated.append(df_help)
    df_1082_prepared = pd.concat(df_separated, axis = 1)
    df_1082_prepared.to_csv(path_aim)

In [3]:
path_origin = '../data/raw/1082ff_monatsbericht_suesswaren_schoko_kaffee.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'
prepare_monatsbericht(path_origin, path_aim)

C:\Users\judit\AppData\Local\Temp\ipykernel_30068\2423498605.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df_1082 = pd.read_csv(path_origin, skiprows =7,


In [4]:
path_origin = '../data/raw/1082_monatsbericht_suesswaren_timeseries.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'
prepare_monatsbericht(path_origin, path_aim)

C:\Users\judit\AppData\Local\Temp\ipykernel_30068\2423498605.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df_1082 = pd.read_csv(path_origin, skiprows =7,


## Einfuhr und Erzeugerpreise

In [5]:
def prepare_GP(path_origin, path_aim, praefix = ''):
    df= pd.read_csv(path_origin, 
                skiprows =6,
                skipfooter = 3, sep = ';'
                )
    for i in range(3, 171, 2):
        zahl = str(i)
        df = df.drop('Unnamed: '+zahl, axis=1)
    df = df.transpose()
    columns = df.iloc[1,:]
    columns.values[0] = 'Month'
    for i in range(1,len(columns)):
        columns.values[i] = praefix + '_' + columns.values[i]
    df.columns = columns
    df = df.drop('Unnamed: 0', axis = 0)
    df = df.drop('Unnamed: 1', axis = 0)
    list_year = []
    for i in range(7):
        list_year = list_year + np.full(12,i+2019).tolist()
    df['Year'] = list_year
    df['Day']=1
    df['Month'] = df['Month'].replace({'Januar':1, 'Februar':2, 'März': 3, 'April':4, 'Mai':5, 
                                   'Juni': 6, 'Juli': 7, 'August': 8, 'September': 9, 'Oktober': 10, 'November': 11, 'Dezember': 12})

    df['date']=pd.to_datetime(df[['Year', 'Month', 'Day']])
    df = df.drop('Month', axis = 1)
    df = df.drop('Year', axis = 1)
    df = df.drop('Day', axis = 1)
    df = df.set_index('date')
    df.to_csv(path_aim)

In [6]:
path_origin = '../data/raw/1082ff_erzeugerpreise_suesswaren_schoko_kaffee.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'
prepare_GP(path_origin, path_aim, 'ErzPr')

C:\Users\judit\AppData\Local\Temp\ipykernel_30068\99283346.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df= pd.read_csv(path_origin,
C:\Users\judit\AppData\Local\Temp\ipykernel_30068\99283346.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Month'] = df['Month'].replace({'Januar':1, 'Februar':2, 'März': 3, 'April':4, 'Mai':5,


In [7]:
path_origin = '../data/raw/1082_1083_einfuhrpreise_suesswarenundkaffee.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'
prepare_GP(path_origin, path_aim, 'EinfPr')

C:\Users\judit\AppData\Local\Temp\ipykernel_30068\99283346.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df= pd.read_csv(path_origin,
C:\Users\judit\AppData\Local\Temp\ipykernel_30068\99283346.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Month'] = df['Month'].replace({'Januar':1, 'Februar':2, 'März': 3, 'April':4, 'Mai':5,
